## Construcción y Entrenamiento del Modelo

In [11]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Función para cargar y procesar imágenes de entrenamiento
def load_and_preprocess_images(data, img_size=(48, 48), path_col='path', label_col='label', filter_labels=None):
    images = []
    labels = []
    for index, row in data.iterrows():
        if filter_labels is None or row[label_col] in filter_labels:
            img_path = row[path_col].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row[label_col])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

In [ ]:
# Función para cargar y procesar imágenes sin etiquetas (para el conjunto de prueba)
def load_and_preprocess_images2(data, img_size=(48, 48)):
    images = []
    for index, row in data.iterrows():
        img_path = f"data/data/images/test/{row['id_img']}.jpg"
        img = load_img(img_path, target_size=img_size, color_mode='grayscale')
        img_array = img_to_array(img)
        images.append(img_array)
    images = np.array(images, dtype='float32') / 255.0
    return images

In [ ]:
# Cargar los datos
train_data = pd.read_csv('data/train_set.csv')
test_data = pd.read_csv('test_set.csv')
sample_submission = pd.read_csv('sample_submision.csv')

In [9]:
# Mostrar las columnas de los DataFrames
print("Columnas de train_set.csv:", train_data.columns)
print("Columnas de test_set.csv:", test_data.columns)

Columnas de train_set.csv: Index(['id_img', 'path', 'label'], dtype='object')
Columnas de test_set.csv: Index(['id_img'], dtype='object')


In [ ]:
# Cargar y procesar imágenes de entrenamiento
train_images, train_labels = load_and_preprocess_images(train_data, path_col=path_col, label_col=label_col, filter_labels=['happy', 'sad'])

In [ ]:

# Codificar etiquetas
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_categorical = to_categorical(train_labels_encoded)

In [ ]:
# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_categorical, test_size=0.2, random_state=42)

In [12]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

In [ ]:
# Definir la red neuronal
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [13]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
  1/303 ━━━━━━━━━━━━━━━━━━━━ 7:34 2s/step - accuracy: 0.8438 - loss: 0.4460

C:\Users\buque\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


303/303 ━━━━━━━━━━━━━━━━━━━━ 17s 50ms/step - accuracy: 0.8168 - loss: 0.3977 - val_accuracy: 0.8620 - val_loss: 0.3155
Epoch 2/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.8310 - loss: 0.3638 - val_accuracy: 0.8563 - val_loss: 0.3275
Epoch 3/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.8341 - loss: 0.3652 - val_accuracy: 0.8686 - val_loss: 0.3080
Epoch 4/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.8431 - loss: 0.3512 - val_accuracy: 0.8604 - val_loss: 0.3109
Epoch 5/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 44ms/step - accuracy: 0.8451 - loss: 0.3334 - val_accuracy: 0.8658 - val_loss: 0.3236
Epoch 6/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.8553 - loss: 0.3184 - val_accuracy: 0.8645 - val_loss: 0.3051
Epoch 7/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.8621 - loss: 0.3213 - val_accuracy: 0.8666 - val_loss: 0.3155
Epoch 8/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.8592 - loss: 0.3265 - val_accurac

In [14]:
# Evaluar el modelo
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_accuracy:.4f}')

76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8961 - loss: 0.2568
Validation accuracy: 0.8905


In [15]:
# Cargar y procesar imágenes de prueba
test_images = load_and_preprocess_images2(test_data)

In [ ]:
# Asegurarse de que las imágenes de prueba tengan la forma correcta
if test_images.ndim == 3:
    test_images = np.expand_dims(test_images, axis=-1)

In [ ]:
# Verificar la forma de las imágenes de prueba
print(f"Forma de las imágenes de prueba: {test_images.shape}")

In [ ]:
# Asegurarse de que no hay imágenes con dimensiones incorrectas
for i, img in enumerate(test_images):
    if img.shape != (48, 48, 1):
        print(f"Imagen en índice {i} tiene una forma incorrecta: {img.shape}")

In [16]:
# Realizar predicciones sobre el conjunto de prueba
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


In [17]:
# Generar el archivo de submission
submission = sample_submission.copy()
submission['label'] = predicted_labels_decoded
submission.to_csv('submission1.csv', index=False)

In [7]:
# Ajustar los nombres de las columnas según sea necesario
path_col = 'path'  # Cambia esto si el nombre de la columna es diferente
label_col = 'label'  # Cambia esto si el nombre de la columna es diferente

Forma de las imágenes de prueba: (7066, 48, 48, 1)
221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


In [18]:
submission

,id_img,label
0,10052,happy
1,10065,sad
2,10079,sad
3,10095,sad
4,10121,sad
...,...,...
7061,9806,sad
7062,9830,happy
7063,9853,sad
7064,9878,happy


In [13]:
# Manipulación de datos
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import zipfile
import Funciones_Kaggle_ReconocimientoFacial as fc

# Carga y preprocesamiento de imágenes
import os
import cv2

# Construcción y entrenamiento del modelo
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [6]:
import zipfile
import pandas as pd
import os

# Descomprimir archivos
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')
with zipfile.ZipFile('train_set.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

# Cargar los datos
train_data = pd.read_csv('data/train_set.csv')
test_data = pd.read_csv('test_set.csv')
sample_submission = pd.read_csv('sample_submision.csv')

# Mostrar las primeras filas de los datos de entrenamiento
train_data.head()

,id_img,path,label
0,0,../data/images/train/angry\0.jpg,angry
1,1,../data/images/train/angry\1.jpg,angry
2,10,../data/images/train/angry\10.jpg,angry
3,10002,../data/images/train/angry\10002.jpg,angry
4,10016,../data/images/train/angry\10016.jpg,angry


In [20]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

def load_and_preprocess_images(data, img_size=(48, 48)):
    images = []
    labels = []
    for index, row in data.iterrows():
        if row['label'] in ['happy', 'sad']:  # Filtrar solo 'happy' y 'sad'
            img_path = row['path'].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row['label'])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

# Cargar y procesar imágenes de entrenamiento
train_images, train_labels = load_and_preprocess_images(train_data)

# Codificar etiquetas
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_categorical = to_categorical(train_labels_encoded)

# Mostrar la forma de las imágenes y etiquetas de entrenamiento
train_images.shape, train_labels_categorical.shape

((12102, 48, 48, 1), (12102, 2))

In [21]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_categorical, test_size=0.2, random_state=42)

In [22]:
# Definir la red neuronal
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

C:\Users\buque\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [24]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.6003 - loss: 0.6707 - val_accuracy: 0.7183 - val_loss: 0.5606
Epoch 2/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.7359 - loss: 0.5228 - val_accuracy: 0.7869 - val_loss: 0.4428
Epoch 3/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.7828 - loss: 0.4497 - val_accuracy: 0.8162 - val_loss: 0.3931
Epoch 4/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.8233 - loss: 0.3780 - val_accuracy: 0.8319 - val_loss: 0.3553
Epoch 5/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8538 - loss: 0.3320 - val_accuracy: 0.8373 - val_loss: 0.3453
Epoch 6/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.8550 - loss: 0.3242 - val_accuracy: 0.8513 - val_loss: 0.3324
Epoch 7/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.8811 - loss: 0.2800 - val_accuracy: 0.8604 - val_loss: 0.3330
Epoch 8/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.8950 - loss: 0.2428 - 

In [25]:
# Evaluar el modelo
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_accuracy:.4f}')

76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8539 - loss: 0.3543
Validation accuracy: 0.8567


In [43]:
# Función para cargar y procesar imágenes sin etiquetas (para el conjunto de prueba)
def load_and_preprocess_images2(data, img_size=(48, 48)):
    images = []
    labels = []
    for index, row in data.iterrows():
        if row['id_img'] in ['happy', 'sad']:  # Filtrar solo 'happy' y 'sad'
            img_path = row['path'].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row['label'])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

In [44]:
# Cargar y procesar imágenes de prueba
test_images = load_and_preprocess_images2(test_data)

In [49]:
# Asegurarse de que las imágenes de prueba tengan la forma correcta
if test_images.ndim == 3:
    test_images = np.expand_dims(test_images, axis=-1)

In [51]:
# Realizar predicciones sobre el conjunto de prueba
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

ValueError: Exception encountered when calling Conv2D.call().

[1mNegative dimension size caused by subtracting 3 from 0 for '{{node sequential_1_1/conv2d_3_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](data, sequential_1_1/conv2d_3_1/convolution/ReadVariableOp)' with input shapes: [2,0,1,1], [3,3,1,32].[0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(2, 0, 1, 1), dtype=float32)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Define image data generators
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator()

In [ ]:
# Create image data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

In [ ]:
# Define the model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: happy and sad
])

In [ ]:
model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    validation_split=0.2, 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[early_stopping])

In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
val_loss, val_accuracy

In [ ]:
test_df = pd.DataFrame({'id_img': [int(os.path.splitext(f)[0]) for f in test_images]})

In [ ]:
test_images = []
test_ids = []
for image_file in test_images:
    image_path = os.path.join(test_dir, image_file)
    img = preprocess_image(image_path)
    if img is not None:
        test_images.append(img)
        test_ids.append(int(os.path.splitext(image_file)[0]))

test_images = np.array(test_images)

In [ ]:
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
label_map = {0: 'sad', 1: 'happy'}
predicted_labels = [label_map[label] for label in predicted_labels]